In [1]:
!pip install nltk
!pip install tensorflow==2.16.1
!pip install flask-ngrok
!pip install pyngrok

In [2]:
!pip install gradio

In [14]:
# Import necessary libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


import tensorflow as tf
print(tf.__version__)
import pickle
import numpy as np
from tensorflow.keras.models import load_model
import json
import random
import gradio as gr

2.16.1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [15]:
from flask import Flask, jsonify
from pyngrok import ngrok
# Load model and data files
from tensorflow.keras.models import load_model
model = load_model('/content/drive/chatbot_model.h5')
intents = json.loads(open('/content/drive/intents.json', encoding="utf8").read())
words = pickle.load(open('/content/drive/words.pkl', 'rb'))
classes = pickle.load(open('/content/drive/classes.pkl', 'rb'))

In [16]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  print("sentence words", sentence_words)
  return sentence_words

In [17]:
def bow(sentence, words, show_details=True):
  #tokenize the pattern
  sentence_words = clean_up_sentence(sentence)
  #bag of words - matrix of N words
  bag = [0]*len(words)

  for s in sentence_words:
    for i,w in enumerate(words):
      if(w==s):
        #assign 1 if current word is in the vocabulary position
        bag[i] = 1
        if show_details:
          print('found in bag: %s' % w )
  return(np.array(bag))

In [18]:
def predict_class(sentence, model):
  #filter out predictions below a threshold
  p = bow(sentence, words, show_details=False)
  print("p = ",p)
  res = model.predict(np.array([p]))[0]
  print("res = ",res)
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  print("results = ", results)
  #sort by strength of probabilty
  results.sort(key = lambda x: x[1], reverse=True)
  result_list = []

  for r in results:
    result_list.append({"intent":classes[r[0]], "probabilty": str(r[1])})

  return result_list

In [19]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  print("tag = ", tag)
  list_of_intents = intents_json['intents']
  print("list of intents = ", list_of_intents)
  for i in list_of_intents:
    if(i['tag']==tag):
      result = random.choice(i['responses'])
      break
  print("final result = ", result)
  return result

In [20]:
def chatbot_response(msg):
    try:
        ints = predict_class(msg, model)
        if ints:
            res = getResponse(ints, intents)
            return res
        else:
            return "I'm sorry, I don't understand."
    except Exception as e:
        print(f"Error: {e}")
        return "ERROR"

In [21]:
# Function to handle new messages and return the updated chat history
def respond(message, history):
    response = chatbot_response(message)
    history.append((message, response))
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(respond, [msg, chatbot], chatbot)
    clear.click(lambda: [], None, chatbot)


In [22]:
# Launch the Gradio interface
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6455a99d4b02e727f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
